In [1]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
import numpy as np
from custom_dataset import Binary_Dataset
from matplotlib import pyplot as plt
from PIL import Image
import os
from inception import inception_v3
from Trainer import train_loop
from torch.optim import Adam
import pandas as pd

%matplotlib inline
torch.cuda.empty_cache()

# Define directories, transformations, and labels


In [2]:
#define relevant paths
labels_dir="Data\list_attr_celeba.csv"
image_dir="Data\img_align_celeba\img_align_celeba"

##############################################define num workers for data laoders here
workers=4

#define composed transforms
composed = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize([299,299]),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])


labels=["Male","Smiling","Wavy_Hair"]
label_number=1


# Load CNN's

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model=inception_v3().to(device)

In [4]:
model.load_state_dict(torch.load('Smiling_12_8.pt'))
model.eval()

running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Mixed_5b): InceptionA(
    (branch1x1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (branch5x5_1): BasicConv2d(
      (conv): Conv2d(192, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (branch5x5_2): BasicConv2d(
      (conv): Conv2d(48, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (branch3x3dbl_1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (branch3x3dbl

# Feature Extractions

In [5]:
# feature extraction for training
#check feature extrcation

# 162770
# 19867
# 19962

bathsize = 35
FE_ds=Binary_Dataset(labels_dir, labels[label_number],image_dir ,transform=composed, partition="Train")
FE_loader=DataLoader(FE_ds,batch_size=bathsize, shuffle=False, num_workers=workers)

device = 'cuda'

FE_data=iter(FE_loader)

#model=model.to(device)

model.eval()

correct=0
train_tested_samples=0
limit=100000


l = ['label']
range_int = range(1,2049)
r = [str(x) for x in range_int]
col = l + r   
px = pd.DataFrame(columns = col)  
px.to_csv('smile2_train.csv', mode='a', index=False, header=True) ##################change csv name
    
    
for data in FE_data:
    sample, target = data
    
    #send to device
    sample=sample.to(device)
    target=target.to(device)
    
    output, extraction = model(sample)
    
#     print(output.shape)
#     print(extraction.shape)
#     print(extraction)
    
    #rediction, extraction = output
        
    target_val = target.cpu().detach().numpy()
    #label_val = np.reshape(target_val, (50,1))
    
#     print(target_val.shape)
    #print(label_val)    
    extract = extraction.cpu().detach().numpy()
    

    data = np.insert(extract, 0, target_val, axis = 1)
#     print(data.shape)
#     print(data)
    data = np.reshape(data, (bathsize, 2049))
 
    px = pd.DataFrame(data) 
    px.to_csv('Male_train.csv', mode='a', index=False, header=False)   ######################csv name 
        
    
    pred=torch.argmax(output,dim=1,keepdim=True)

    correct+=torch.eq(pred,target.view_as(pred)).sum()
    train_tested_samples+=pred.size()[0]
    if train_tested_samples>limit:
        break

    print(correct)
    print(train_tested_samples)

#print precision
print(correct.item()/train_tested_samples)
print(train_tested_samples)

RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 8.00 GiB total capacity; 2.50 GiB already allocated; 0 bytes free; 2.54 GiB reserved in total by PyTorch)

In [ ]:

#validation feature extraction 

# 162770
# 19867
# 19962

bathsize = 35
FE_ds=Binary_Dataset(labels_dir, labels[label_number],image_dir ,transform=composed, partition="Val")
FE_loader=DataLoader(FE_ds,batch_size=bathsize, shuffle=False, num_workers=workers)

device = 'cuda'

FE_data=iter(FE_loader)

#model=model.to(device)

model.eval()

correct=0
val_tested_samples=0
limit=19000


l = ['label']
range_int = range(1,2049)
r = [str(x) for x in range_int]
col = l + r   
px = pd.DataFrame(columns = col)  
px.to_csv('male_val_img.csv', mode='a', index=False, header=True)  ##################change csv name
    
    
for data in FE_data:
    sample, target = data
    
    #send to device
    sample=sample.to(device)
    target=target.to(device)
    
    output, extraction = model(sample)
    
#     print(output.shape)
#     print(extraction.shape)
#     print(extraction)
    
    #rediction, extraction = output
        
    target_val = target.cpu().detach().numpy()
    #label_val = np.reshape(target_val, (50,1))
    
#     print(target_val.shape)
    #print(label_val)    
    extract = extraction.cpu().detach().numpy()
    

    data = np.insert(extract, 0, target_val, axis = 1)
#     print(data.shape)
#     print(data)
    data = np.reshape(data, (bathsize, 2049))
 
    px = pd.DataFrame(data) 

    px.to_csv('male_val_img.csv', mode='a', index=False, header=False)  ##################change csv name  
        
    
    pred=torch.argmax(output,dim=1,keepdim=True)

    correct+=torch.eq(pred,target.view_as(pred)).sum()
    val_tested_samples+=pred.size()[0]
    if val_tested_samples>limit:
        break

    print(correct)
    print(val_tested_samples)

#print precision
print(correct.item()/val_tested_samples)
print(val_tested_samples)

In [5]:
#test feature extraction 

bathsize = 5
FE_ds=Binary_Dataset(labels_dir, labels[label_number],image_dir ,transform=composed, partition="Test")
FE_loader=DataLoader(FE_ds,batch_size=bathsize, shuffle=False, num_workers=workers)

device = 'cuda'

FE_data=iter(FE_loader)

#model=model.to(device)

model.eval()

correct=0
test_tested_samples=0
limit=19000


l = ['label']
range_int = range(1,2049)
r = [str(x) for x in range_int]
col = l + r   
px = pd.DataFrame(columns = col)  
px.to_csv('smile2_test_img.csv', mode='a', index=False, header=True)    ##################change csv name
    
    
for data in FE_data:
    sample, target = data
    
    #send to device
    sample=sample.to(device)
    target=target.to(device)
    
    output, extraction = model(sample)
    
#     print(output.shape)
#     print(extraction.shape)
#     print(extraction)
    
    #rediction, extraction = output
        
    target_val = target.cpu().detach().numpy()
    #label_val = np.reshape(target_val, (50,1))
    
#     print(target_val.shape)
    #print(label_val)    
    extract = extraction.cpu().detach().numpy()
    

    data = np.insert(extract, 0, target_val, axis = 1)
#     print(data.shape)
#     print(data)
    val_data = np.reshape(data, (bathsize, 2049))
 
    px = pd.DataFrame(val_data) 

    px.to_csv('smile2_test_img.csv', mode='a', index=False, header=False)   ##################change csv name
        
    
    pred=torch.argmax(output,dim=1,keepdim=True)

    correct+=torch.eq(pred,target.view_as(pred)).sum()
    test_tested_samples+=pred.size()[0]
    if test_tested_samples>limit:
        break

    print(correct)
    print(test_tested_samples)


#print precision
print(correct.item()/test_tested_samples)
print(test_tested_samples)

.0699,  ..., 0.1397, 0.0031, 0.0532],
        [0.0446, 0.0164, 0.0189,  ..., 0.0205, 0.1367, 0.0669],
        [0.0000, 0.0148, 0.0055,  ..., 0.0000, 0.0000, 0.0054]],
       device='cuda:0')
tensor(17448, device='cuda:0')
18775
tensor([[0.1120, 0.0431, 0.0716,  ..., 0.1403, 0.0020, 0.0584],
        [0.1462, 0.0467, 0.1022,  ..., 0.2135, 0.0134, 0.0974],
        [0.1033, 0.0668, 0.1266,  ..., 0.1627, 0.4529, 0.2603],
        [0.0446, 0.0222, 0.0250,  ..., 0.0290, 0.1449, 0.0648],
        [0.2637, 0.0699, 0.1654,  ..., 0.3794, 0.0022, 0.1472]],
       device='cuda:0')
tensor(17453, device='cuda:0')
18780
tensor([[0.0677, 0.0368, 0.0488,  ..., 0.0645, 0.0178, 0.0325],
        [0.0014, 0.0057, 0.0022,  ..., 0.0007, 0.0007, 0.0025],
        [0.2698, 0.0655, 0.1785,  ..., 0.3875, 0.0106, 0.1658],
        [0.1038, 0.0503, 0.0755,  ..., 0.1257, 0.0005, 0.0636],
        [0.0577, 0.0175, 0.0070,  ..., 0.0146, 0.1614, 0.0517]],
       device='cuda:0')
tensor(17458, device='cuda:0')
18785
tensor([

In [ ]:
# this is just used to check the output 

df_attr_train=pd.read_csv(labels_dir, nrows = train_tested_samples, usecols=["image_id",labels[label_number]]) 
# df_attr_val=pd.read_csv(labels_dir, skiprows = [i for i in range(1,162771)], nrows = val_tested_samples, usecols=["image_id",labels[label_number]])
# df_attr_test=pd.read_csv(labels_dir, skiprows = [i for i in range(1,182638)], nrows = test_tested_samples, usecols=["image_id",labels[label_number]])

# #change -1 to 0 in csv, remove this if not needed
df_attr_train.replace(-1,0,inplace=True)
#df_attr_val.replace(-1,0,inplace=True)
# df_attr_test.replace(-1,0,inplace=True)



In [ ]:
df_attr_train.info()

In [ ]:
df_attr_train.head(n=37)

In [ ]:
df_attr_train.tail(n=15)

In [7]:
# df_attr_train=pd.read_csv(labels_dir, nrows = 100030, usecols=["image_id"]) 
# train_imgs = df_attr_train.to_numpy()
# traindata = pd.read_csv('smile_train_img.csv')
# traindata.insert(0, 'img_id', train_imgs)
# traindata.to_csv('smile_train.csv', mode='a', index=False)

# df_attr_val=pd.read_csv(labels_dir, skiprows = [i for i in range(1,162771)], nrows = val_tested_samples, usecols=["image_id"])
# val_imgs = df_attr_val.to_numpy()
# valdata = pd.read_csv('smile_val_img.csv')
# #valdata.insert(0, 'label_from_csv', val_imgs[:,1])
# valdata.insert(0, 'img_id', val_imgs[:,0])
# valdata.to_csv('smile_val.csv', mode='a', index=False)


df_attr_test=pd.read_csv(labels_dir, skiprows = [i for i in range(1,182638)], nrows = test_tested_samples, usecols=["image_id"])
test_imgs = df_attr_test.to_numpy()
testdata = pd.read_csv('smile2_test_img.csv')
testdata.insert(0, 'img name', test_imgs)
testdata.to_csv('smile2_test_img.csv', mode='a', index=False)

In [ ]:
traindata = pd.read_csv('smile_train_img.csv')

In [ ]:
traindata.info()